<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Week 3 Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Scraping Data with Beautiful Soup</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [57]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import wget

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Scraping Data with Beautiful Soup

Web scraping of Wikipedia page provided by assignment is done using Beautiful Soup.

In [196]:
from bs4 import BeautifulSoup
import requests

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html.parser')

#### Load and explore the data

Next, let's load the data.

It is found out that the table in wikipedia page has a certain tag, the table section in the soup will be assign to an element

In [71]:
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<

All elements are then list up accordingly with the 'td' header

In [193]:
links=My_table.find_all('td')
print(links)

[<td>M1A
</td>, <td>Not assigned
</td>, <td>
</td>, <td>M2A
</td>, <td>Not assigned
</td>, <td>
</td>, <td>M3A
</td>, <td>North York
</td>, <td>Parkwoods
</td>, <td>M4A
</td>, <td>North York
</td>, <td>Victoria Village
</td>, <td>M5A
</td>, <td>Downtown Toronto
</td>, <td>Regent Park, Harbourfront
</td>, <td>M6A
</td>, <td>North York
</td>, <td>Lawrence Manor, Lawrence Heights
</td>, <td>M7A
</td>, <td>Downtown Toronto
</td>, <td>Queen's Park, Ontario Provincial Government
</td>, <td>M8A
</td>, <td>Not assigned
</td>, <td>
</td>, <td>M9A
</td>, <td>Etobicoke
</td>, <td>Islington Avenue, Humber Valley Village
</td>, <td>M1B
</td>, <td>Scarborough
</td>, <td>Malvern, Rouge
</td>, <td>M2B
</td>, <td>Not assigned
</td>, <td>
</td>, <td>M3B
</td>, <td>North York
</td>, <td>Don Mills
</td>, <td>M4B
</td>, <td>East York
</td>, <td>Parkview Hill, Woodbine Gardens
</td>, <td>M5B
</td>, <td>Downtown Toronto
</td>, <td>Garden District, Ryerson
</td>, <td>M6B
</td>, <td>North York
</td>, <td>Glenc

Before further processing, the '\n' in each element in string is replace with '' for the convenience of further processing

In [198]:
table=[]
for link in links:
    table.append(link.get_text())
    
newtable=[]
for item in table:
    newitem=item.replace('\n','')
    newtable.append(newitem)

As the table has 3 columns, the intention was to create 3 dataframe and concat it separately afterwards for further processing

In [199]:
col1=[]
col2=[]
col3=[]
n=0
for index, item in enumerate(newtable):
    
    if n==0:
        col1.append(item)
        n+=1
    elif n==1:
        col2.append(item)
        n+=1
    elif n==2:
        col3.append(item)
        n=0        

The columns will be named, unassigned borough to be removed and the index will be reset 

In [213]:
col1a=pd.DataFrame(data=col1,columns=['Postal_Code'])
col2a=pd.DataFrame(data=col2,columns=['Borough'])
col3a=pd.DataFrame(data=col3,columns=['Neighbourhood'])
combi=pd.concat([col1a,col2a,col3a],axis=1)
combi=combi[combi.Borough!='Not assigned']
combi=combi.sort_values('Postal_Code')
combi=combi.reset_index(drop=True)
combi

,Postal_Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


The table is checked for any duplicated postal code to merge afterwards

In [214]:
combi[combi.duplicated('Postal_Code')]

,Postal_Code,Borough,Neighbourhood


As requested the shape of the table is as mention

In [215]:
combi.shape

(103, 3)

In [219]:
table2=pd.read_csv('Geospatial_Coordinates.csv')
table2

,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [220]:
ans=pd.merge(combi,table2,on=['Postal_Code'])
ans

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [225]:
ans.sort_values("Borough")

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
64,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [222]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of New York with neighborhoods superimposed on top.

In [224]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(ans['Latitude'], ans['Longitude'], ans['Borough'], ans['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [227]:
eastyork_data = ans[ans['Borough'] == 'East York'].reset_index(drop=True)
eastyork_data.head()

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude
0,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
1,M4C,East York,Woodbine Heights,43.695344,-79.318389
2,M4G,East York,Leaside,43.709060,-79.363452
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372
4,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106


Let's get the geographical coordinates of Manhattan.

In [312]:
address = 'East York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East York are 43.699971000000005, -79.33251996261595.


let's visualizat East York the neighborhoods in it.

In [313]:
# create map of East York using latitude and longitude values
map_eastyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(eastyork_data['Latitude'], eastyork_data['Longitude'], eastyork_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_eastyork)  
    
map_eastyork

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [232]:
CLIENT_ID = '2PXGMUJF0IOVFDI2WCRNW5BCL325Y3GRQNRJMFKH4JWLDE4Z' # your Foursquare ID
CLIENT_SECRET = 'C4WKMAS3XK2KUGN42N2D5WNLN1LOH3VEB4Z1V44GX0JATRZD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2PXGMUJF0IOVFDI2WCRNW5BCL325Y3GRQNRJMFKH4JWLDE4Z
CLIENT_SECRET:C4WKMAS3XK2KUGN42N2D5WNLN1LOH3VEB4Z1V44GX0JATRZD


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [235]:
eastyork_data.loc[0, 'Neighbourhood']

'Parkview Hill, Woodbine Gardens'

Get the neighborhood's latitude and longitude values.

In [236]:
neighborhood_latitude = eastyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = eastyork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = eastyork_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkview Hill, Woodbine Gardens are 43.7063972, -79.309937.


#### Now, let's get the top 100 venues that are in Parkview Hill, Woodbine Gardens within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [237]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2PXGMUJF0IOVFDI2WCRNW5BCL325Y3GRQNRJMFKH4JWLDE4Z&client_secret=C4WKMAS3XK2KUGN42N2D5WNLN1LOH3VEB4Z1V44GX0JATRZD&v=20180605&ll=43.7063972,-79.309937&radius=500&limit=100'

Send the GET request and examine the resutls

In [238]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ece841329ce6a001bcd70c5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': "O'Connor - Parkview",
  'headerFullLocation': "O'Connor - Parkview, Toronto",
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 43.7108972045, 'lng': -79.30372360313615},
   'sw': {'lat': 43.701897195499996, 'lng': -79.31615039686386}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b5a3842f964a52023b528e3',
       'name': 'Jawny Bakers',
       'location': {'address': "804 O'Connor Dr",
        'crossStreet': 'St Clair E',
        'lat': 43.705782646822,
        'lng': -79.31291304477831,
        'labeledLatLngs':

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [239]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [240]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Jawny Bakers,Gastropub,43.705783,-79.312913
1,East York Gymnastics,Gym / Fitness Center,43.710654,-79.309279
2,Shoppers Drug Mart,Pharmacy,43.705933,-79.312825
3,TD Canada Trust,Bank,43.705740,-79.312270
4,Pizza Pizza,Pizza Place,43.705159,-79.313130


And how many venues were returned by Foursquare?

In [241]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in East York

In [255]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [256]:
# type your answer here

eastyork_venues = getNearbyVenues(names=eastyork_data['Neighbourhood'],
                                   latitudes=eastyork_data['Latitude'],
                                   longitudes=eastyork_data['Longitude']
                                  )



Parkview Hill, Woodbine Gardens
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)


#### Let's check the size of the resulting dataframe

In [245]:
print(eastyork_venues.shape)
eastyork_venues.head()

(74, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Jawny Bakers,43.705783,-79.312913,Gastropub
1,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,East York Gymnastics,43.710654,-79.309279,Gym / Fitness Center
2,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Shoppers Drug Mart,43.705933,-79.312825,Pharmacy
3,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,TD Canada Trust,43.705740,-79.312270,Bank
4,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Pizza Pizza,43.705159,-79.313130,Pizza Place


Let's check how many venues were returned for each neighborhood

In [246]:
eastyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"East Toronto, Broadview North (Old East York)",4,4,4,4,4,4
Leaside,33,33,33,33,33,33
"Parkview Hill, Woodbine Gardens",10,10,10,10,10,10
Thorncliffe Park,19,19,19,19,19,19
Woodbine Heights,8,8,8,8,8,8


#### Let's find out how many unique categories can be curated from all the returned venues

In [247]:
print('There are {} uniques categories.'.format(len(eastyork_venues['Venue Category'].unique())))

There are 46 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [251]:
# one hot encoding
eastyork_onehot = pd.get_dummies(eastyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
eastyork_onehot['Neighbourhood'] = eastyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [eastyork_onehot.columns[-1]] + list(eastyork_onehot.columns[:-1])
eastyork_onehot = eastyork_onehot[fixed_columns]

eastyork_onehot.head()

,Neighbourhood,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Coffee Shop,Convenience Store,Cosmetics Shop,Curling Ice,Department Store,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Indian Restaurant,Intersection,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Video Store,Warehouse Store,Yoga Studio
0,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Parkview Hill, Woodbine Gardens",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [252]:
eastyork_onehot.shape

(3077, 328)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [257]:
eastyork_grouped = eastyork_onehot.groupby('Neighbourhood').mean().reset_index()
eastyork_grouped

,Neighbourhood,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Coffee Shop,Convenience Store,Cosmetics Shop,Curling Ice,Department Store,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gas Station,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Indian Restaurant,Intersection,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Video Store,Warehouse Store,Yoga Studio
0,"East Toronto, Broadview North (Old East York)",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.250000,0.25,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
1,Leaside,0.0,0.030303,0.060606,0.030303,0.030303,0.030303,0.030303,0.060606,0.000,0.090909,0.00,0.000,0.000,0.030303,0.030303,0.000000,0.030303,0.000000,0.030303,0.060606,0.000000,0.0,0.030303,0.030303,0.0,0.000000,0.0,0.030303,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.030303,0.030303,0.030303,0.000,0.030303,0.090909,0.030303,0.030303,0.030303,0.000,0.000000,0.000000
2,"Parkview Hill, Woodbine Gardens",0.1,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.1,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.200000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000
3,Thorncliffe Park,0.0,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.052632,0.000,0.052632,0.00,0.000,0.000,0.000000,0.000000,0.052632,0.000000,0.052632,0.000000,0.000000,0.052632,0.0,0.052632,0.052632,0.0,0.105263,0.0,0.000000,0.000000,0.052632,0.052632,0.000000,0.052632,0.052632,0.052632,0.052632,0.000000,0.000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000,0.052632,0.052632
4,Woodbine Heights,0.0,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.00,0.125,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.125000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000


#### Let's confirm the new size

In [259]:
eastyork_grouped.shape

(5, 47)

#### Let's print each neighborhood along with the top 5 most common venues

In [266]:
num_top_venues = 5

for hood in eastyork_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = eastyork_grouped[eastyork_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----East Toronto, Broadview North (Old East York)----
                venue  freq
0                Park  0.25
1         Coffee Shop  0.25
2   Convenience Store  0.25
3         Pizza Place  0.25
4  Athletics & Sports  0.00


----Leaside----
                    venue  freq
0             Coffee Shop  0.09
1     Sporting Goods Shop  0.09
2                    Bank  0.06
3            Burger Joint  0.06
4  Furniture / Home Store  0.06


----Parkview Hill, Woodbine Gardens----
                  venue  freq
0           Pizza Place   0.2
1    Athletics & Sports   0.1
2          Intersection   0.1
3             Gastropub   0.1
4  Fast Food Restaurant   0.1


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.11
1                Gym  0.05
2     Discount Store  0.05
3    Warehouse Store  0.05
4        Supermarket  0.05


----Woodbine Heights----
         venue  freq
0  Curling Ice  0.12
1   Beer Store  0.12
2  Video Store  0.12
3         Park  0.12
4     Bus Stop  0.12




#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [286]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [301]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = eastyork_grouped['Neighbourhood']

for ind in np.arange(eastyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(eastyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"East Toronto, Broadview North (Old East York)",Pizza Place,Park,Coffee Shop,Convenience Store,Cosmetics Shop,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Discount Store
1,Leaside,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Bank,Burger Joint,Grocery Store,Pet Store,Department Store,Fish & Chips Shop,Gym
2,"Parkview Hill, Woodbine Gardens",Pizza Place,Breakfast Spot,Fast Food Restaurant,Gym / Fitness Center,Intersection,Pharmacy,Athletics & Sports,Gastropub,Bank,Burger Joint
3,Thorncliffe Park,Indian Restaurant,Yoga Studio,Pharmacy,Bank,Burger Joint,Coffee Shop,Discount Store,Fast Food Restaurant,Gas Station,Warehouse Store
4,Woodbine Heights,Park,Video Store,Beer Store,Skating Rink,Bus Stop,Curling Ice,Pharmacy,Cosmetics Shop,Yoga Studio,Fish & Chips Shop


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [302]:
# set number of clusters
kclusters = 3

eastyork_grouped_clustering = eastyork_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(eastyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 1, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [303]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

eastyork_merged = eastyork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
eastyork_merged = eastyork_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

eastyork_merged.head() # check the last columns!

,Postal_Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,Pizza Place,Breakfast Spot,Fast Food Restaurant,Gym / Fitness Center,Intersection,Pharmacy,Athletics & Sports,Gastropub,Bank,Burger Joint
1,M4C,East York,Woodbine Heights,43.695344,-79.318389,0,Park,Video Store,Beer Store,Skating Rink,Bus Stop,Curling Ice,Pharmacy,Cosmetics Shop,Yoga Studio,Fish & Chips Shop
2,M4G,East York,Leaside,43.709060,-79.363452,1,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Bank,Burger Joint,Grocery Store,Pet Store,Department Store,Fish & Chips Shop,Gym
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372,1,Indian Restaurant,Yoga Studio,Pharmacy,Bank,Burger Joint,Coffee Shop,Discount Store,Fast Food Restaurant,Gas Station,Warehouse Store
4,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,2,Pizza Place,Park,Coffee Shop,Convenience Store,Cosmetics Shop,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Discount Store


Finally, let's visualize the resulting clusters

In [315]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(eastyork_merged['Latitude'], eastyork_merged['Longitude'], eastyork_merged['Neighbourhood'], eastyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

#### Cluster 1

In [305]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 0, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East York,0,Park,Video Store,Beer Store,Skating Rink,Bus Stop,Curling Ice,Pharmacy,Cosmetics Shop,Yoga Studio,Fish & Chips Shop


#### Cluster 2

In [306]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 1, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East York,1,Pizza Place,Breakfast Spot,Fast Food Restaurant,Gym / Fitness Center,Intersection,Pharmacy,Athletics & Sports,Gastropub,Bank,Burger Joint
2,East York,1,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Bank,Burger Joint,Grocery Store,Pet Store,Department Store,Fish & Chips Shop,Gym
3,East York,1,Indian Restaurant,Yoga Studio,Pharmacy,Bank,Burger Joint,Coffee Shop,Discount Store,Fast Food Restaurant,Gas Station,Warehouse Store


#### Cluster 3

In [307]:
eastyork_merged.loc[eastyork_merged['Cluster Labels'] == 2, eastyork_merged.columns[[1] + list(range(5, eastyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East York,2,Pizza Place,Park,Coffee Shop,Convenience Store,Cosmetics Shop,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Discount Store


<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).